In [57]:
import numpy as np
import scipy.constants as spc
import matplotlib.pyplot as plt

In [58]:
class Mag_Dipole():
    def __init__(self, m):
        self.m = m # initialize the dipole moment

    # def visualize(self):
    #     return

    def get_field(self, r, version = 'vector'):
        
        r = r.reshape(max(r.shape[0], 1), r.shape[-1]) # make sure the position vector is the correct shape for vectorization (must be 2D!)
        
        B_field = np.array((spc.mu_0/(4*np.pi))*(3*r*(np.dot(r, m).reshape(r.shape[0],1))/(np.linalg.norm(r, axis = 1).reshape(r.shape[0],1)**5) - m/(np.linalg.norm(r, axis = 1).reshape(r.shape[0],1)**3))) # calc the B-field

        if version == 'scalar':
            return np.linalg.norm(B_field, axis = 1).reshape(B_field.shape[0],1)
        elif version == 'vector':
            return B_field
        else:
            raise Exception("'%s is not a valid version, please use either 'scalar' or 'vector''" % version)
        

    
    def gen_data(self, r_dist, num_data_pts, version = 'vector'):
                
        r_list = r_dist*np.random.rand(num_data_pts) # generate random radial distances within specified radius
        theta_list = 2*np.pi*np.random.rand(num_data_pts) # generate random polar angles
        phi_list = np.pi*np.random.rand(num_data_pts) # generate random azimuthal angles

        x_list = r_list*np.sin(phi_list)*np.cos(theta_list) # switch to cartesian
        y_list = r_list*np.sin(phi_list)*np.sin(theta_list) # switch to cartesian
        z_list = r_list*np.cos(phi_list) # switch to cartesian

        xyz = np.stack((x_list, y_list, z_list), axis=-1) # group data points together in their own arrays (this is position vector from dipole to measurement)

        B_fields = self.get_field(xyz, version)
        
        label = np.hstack((xyz, np.repeat(self.m.reshape(1,3), len(xyz), axis = 0))) # keep track of the labels corresponding to the B-field

        return B_fields, label

In [95]:
r_dist = 5
num_data_pts = 3
num_sensors = 2

r_list = r_dist*np.random.rand(num_sensors, num_data_pts) # generate random radial distances within specified radius
theta_list = 2*np.pi*np.random.rand(num_sensors, num_data_pts) # generate random polar angles
phi_list = np.pi*np.random.rand(num_sensors, num_data_pts) # generate random azimuthal angles

x_list = r_list*np.sin(phi_list)*np.cos(theta_list) # switch to cartesian
y_list = r_list*np.sin(phi_list)*np.sin(theta_list) # switch to cartesian
z_list = r_list*np.cos(phi_list) # switch to cartesian

xyz = np.stack((x_list, y_list, z_list), axis=-1) # group data points together in their own arrays (this is position vector from dipole to measurement)
print(xyz)
print()

xyz = np.einsum('ijk->jik', xyz).reshape(num_data_pts, num_sensors*3)

print(xyz)

M = np.random.randint(5, size = (num_data_pts, 3))

print(M)
print()

label = np.hstack((xyz,M))
print(label)
print()

M = np.repeat(M[None,:], num_sensors, axis=0)

print()
print(M)



[[[-0.41185625 -0.34732055  0.92706507]
  [-0.6933657   0.29837734 -1.95482338]
  [-0.21782123 -0.00518482 -0.62805778]]

 [[ 0.07732601  0.17092187 -0.7615422 ]
  [-0.52425827  0.59086722 -4.08240678]
  [ 0.06238475 -0.13172196  4.49803432]]]

[[-0.41185625 -0.34732055  0.92706507  0.07732601  0.17092187 -0.7615422 ]
 [-0.6933657   0.29837734 -1.95482338 -0.52425827  0.59086722 -4.08240678]
 [-0.21782123 -0.00518482 -0.62805778  0.06238475 -0.13172196  4.49803432]]
[[4 4 0]
 [3 0 0]
 [2 2 4]]

[[-0.41185625 -0.34732055  0.92706507  0.07732601  0.17092187 -0.7615422
   4.          4.          0.        ]
 [-0.6933657   0.29837734 -1.95482338 -0.52425827  0.59086722 -4.08240678
   3.          0.          0.        ]
 [-0.21782123 -0.00518482 -0.62805778  0.06238475 -0.13172196  4.49803432
   2.          2.          4.        ]]


[[[4 4 0]
  [3 0 0]
  [2 2 4]]

 [[4 4 0]
  [3 0 0]
  [2 2 4]]]


In [65]:
# print(M)
# print(xyz)

B_field = np.array((spc.mu_0/(4*np.pi))*((3*xyz*np.einsum('ijk,ijk->ij', M, xyz).reshape(xyz.shape[0], xyz.shape[1], 1))/(np.linalg.norm(xyz, axis = 2).reshape(xyz.shape[0], xyz.shape[1], 1)**5) - M/(np.linalg.norm(xyz, axis = 2).reshape(xyz.shape[0], xyz.shape[1], 1)**3))) # row-wise dot product
print(B_field)
print()
print()
# print(B_field.reshape(num_data_pts, num_sensors, 3))
# print()
# print()
print(np.einsum('ijk->jik', B_field))

[[[ 1.15733370e-04 -9.84860196e-05  2.74625335e-04]
  [ 1.77906476e-10  1.06144794e-10  1.13177810e-08]
  [-7.47177304e-05  7.59381684e-08 -1.69968275e-05]]

 [[ 3.92680243e-08  2.00089084e-07 -1.99296313e-07]
  [ 4.66800132e-09  1.42195558e-08  3.10057217e-09]
  [-1.38094295e-08  4.81219829e-10 -7.57878911e-09]]]


[[[ 1.15733370e-04 -9.84860196e-05  2.74625335e-04]
  [ 3.92680243e-08  2.00089084e-07 -1.99296313e-07]]

 [[ 1.77906476e-10  1.06144794e-10  1.13177810e-08]
  [ 4.66800132e-09  1.42195558e-08  3.10057217e-09]]

 [[-7.47177304e-05  7.59381684e-08 -1.69968275e-05]
  [-1.38094295e-08  4.81219829e-10 -7.57878911e-09]]]


In [66]:
r = np.array([[5,0,0]])
m = np.array([0,0,1*np.pi*1**2])
# dipole = Mag_Dipole(m)
# print(dipole.get_field(r, version = 'scalar'))
# print()
# print(dipole.get_field(r, version = 'vector'))

B_field = np.array((spc.mu_0/(4*np.pi))*(3*r*(np.dot(r, m).reshape(r.shape[0],1))/(np.linalg.norm(r, axis = 1).reshape(r.shape[0],1)**5) - m/(np.linalg.norm(r, axis = 1).reshape(r.shape[0],1)**3))) # calc the B-field
B_field

array([[ 0.00000000e+00,  0.00000000e+00, -2.51327412e-09]])

In [205]:
def generate_data(num_data_pts, num_sensors, version = 'vector'):

    r_dist = 5
    # num_data_pts = 3
    # num_sensors = 2

    r_list = r_dist*np.random.rand(num_sensors, num_data_pts) # generate random radial distances within specified radius
    theta_list = 2*np.pi*np.random.rand(num_sensors, num_data_pts) # generate random polar angles
    phi_list = np.pi*np.random.rand(num_sensors, num_data_pts) # generate random azimuthal angles

    x_list = r_list*np.sin(phi_list)*np.cos(theta_list) # switch to cartesian
    y_list = r_list*np.sin(phi_list)*np.sin(theta_list) # switch to cartesian
    z_list = r_list*np.cos(phi_list) # switch to cartesian

    xyz = np.stack((x_list, y_list, z_list), axis=-1) # group data points together in their own arrays (this is position vector from dipole to measurement)
    # print(xyz)

    # print()

    M = np.random.randint(5, size = (num_data_pts, 3))

    xyz_label = np.einsum('ijk->jik', xyz).reshape(num_data_pts, num_sensors*3)

    M_label = np.random.randint(5, size = (num_data_pts, 3))

    label = np.hstack((xyz_label,M_label))


    M = np.repeat(M[None,:], num_sensors, axis=0)

    B_field = np.array((spc.mu_0/(4*np.pi))*((3*xyz*np.einsum('ijk,ijk->ij', M, xyz).reshape(xyz.shape[0], xyz.shape[1], 1))/(np.linalg.norm(xyz, axis = 2).reshape(xyz.shape[0], xyz.shape[1], 1)**5) - M/(np.linalg.norm(xyz, axis = 2).reshape(xyz.shape[0], xyz.shape[1], 1)**3))) # row-wise dot product



    if version == 'scalar':
        B_field = np.linalg.norm(B_field, axis = 1)
        B_field = np.einsum('ij->ji', B_field).reshape(num_data_pts, num_sensors)
        return B_field, label
    
    elif version == 'vector':
        B_field = np.einsum('ijk->jik', B_field).reshape(num_data_pts, num_sensors*3)
        return B_field, label
    
    else:
        raise Exception("'%s is not a valid version, please use either 'scalar' or 'vector''" % version)






    r_dist = 5 # NEED TO CHANGE

    r_list = r_dist*np.random.rand(num_data_pts) # generate random radial distances within specified radius
    theta_list = 2*np.pi*np.random.rand(num_data_pts) # generate random polar angles
    phi_list = np.pi*np.random.rand(num_data_pts) # generate random azimuthal angles

    x_list = r_list*np.sin(phi_list)*np.cos(theta_list) # switch to cartesian
    y_list = r_list*np.sin(phi_list)*np.sin(theta_list) # switch to cartesian
    z_list = r_list*np.cos(phi_list) # switch to cartesian

    xyz = np.stack((x_list, y_list, z_list), axis=-1) # group data points together in their own arrays (this is position vector from dipole to measurement)

    m_dist = 5 # NEED TO CHANGE
    M = np.random.randint(5, size=(num_data_pts, 3))


    B_field = np.array((spc.mu_0/(4*np.pi))*((3*xyz*(np.einsum('ij,ij->i', M, xyz)).reshape(xyz.shape[0],1))/(np.linalg.norm(xyz, axis = 1).reshape(xyz.shape[0],1)**5) - M/(np.linalg.norm(xyz, axis = 1).reshape(xyz.shape[0],1)**3))) # calc the B-field

    label = np.hstack((xyz, M)) # keep track of the labels corresponding to the B-field

    if version == 'scalar':
        return np.linalg.norm(B_field, axis = 1).reshape(B_field.shape[0],1), label
    elif version == 'vector':
        return B_field, label
    else:
        raise Exception("'%s is not a valid version, please use either 'scalar' or 'vector''" % version)


In [220]:
data, labels = generate_data(num_data_pts = 1000, num_sensors = 5, version = 'vector')
# print(data[0], labels[0])
print(data, labels)

[[ 5.84145016e-07 -8.96485122e-08  3.87572184e-07 ... -2.11670185e-08
   6.59115829e-08 -1.39731263e-08]
 [-2.76440454e-09 -4.23028398e-09  3.35158061e-09 ... -8.05857906e-08
  -2.19293530e-08 -2.55897013e-08]
 [ 1.59270386e-09 -5.61227798e-09  2.69718761e-09 ... -2.86409196e-07
  -3.47425811e-08 -5.36887820e-07]
 ...
 [-2.86607344e-07 -2.36186662e-06 -2.31416116e-06 ...  1.52757367e-08
  -4.29809538e-09 -9.29699306e-10]
 [ 2.18057751e-04 -2.63634271e-04  2.20213116e-04 ...  2.52930626e-09
  -1.37977116e-09 -2.23110157e-09]
 [ 8.55766563e-09 -6.73049663e-09  4.75047230e-09 ... -3.45552397e-09
  -7.85828465e-09  1.29198877e-08]] [[-7.01088708e-01 -2.06798240e-01 -6.98108049e-01 ...  4.00000000e+00
   2.00000000e+00  0.00000000e+00]
 [ 4.06107385e-01 -7.66205096e-01  4.42496604e+00 ...  2.00000000e+00
   3.00000000e+00  3.00000000e+00]
 [-8.93949204e-01  1.13023808e+00 -3.53367987e+00 ...  4.00000000e+00
   0.00000000e+00  1.00000000e+00]
 ...
 [ 1.72262919e-01  4.90988630e-02 -3.8222396

# Build a NN

In [221]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import tensorflow as tf

In [229]:
model = Sequential()
model.add(Dense(2048, input_shape=(len(data[0]),), activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(len(labels[0])))

def custom_loss(y_pred, y_true):

    xyz_pred, xyz_true = y_pred[:-3], y_true[:-3]
    m_pred, m_true = y_pred[-3:], y_true[-3:]

    res = tf.reduce_mean(tf.add(tf.square(tf.subtract(xyz_pred,xyz_true)), tf.square(tf.subtract(xyz_pred,xyz_true))))

    return res

model.compile(optimizer='adam', loss=custom_loss, metrics=['accuracy'])


In [230]:
model.fit(data, labels, epochs=5, batch_size=10)

Epoch 1/5
100/100 [==============================] - 6s 53ms/step - loss: 22782.4727 - accuracy: 0.1790
Epoch 2/5
100/100 [==============================] - 6s 59ms/step - loss: 2656.6560 - accuracy: 0.1730
Epoch 3/5
100/100 [==============================] - 6s 59ms/step - loss: 231280.3438 - accuracy: 0.1660
Epoch 4/5
100/100 [==============================] - 6s 57ms/step - loss: 4295.0620 - accuracy: 0.1550
Epoch 5/5
100/100 [==============================] - 6s 60ms/step - loss: 12059.2178 - accuracy: 0.1980


In [231]:

test_x, test_y = generate_data(num_data_pts = 1, num_sensors = 5, version = 'vector')
print(test_x)

[[-8.90140221e-09  4.83311129e-09 -5.48396433e-09 -2.59237684e-08
  -8.75565191e-08  4.17284458e-08 -6.36984472e-09 -9.32435772e-09
   9.46064334e-09 -4.56674325e-07 -4.55545985e-09 -2.73329435e-07
  -2.44289875e-07 -5.22300884e-07 -5.08108993e-08]]


In [232]:
model.predict(test_x)

1/1 [==============================] - 0s 48ms/step


array([[ 0.04989507, -0.12470385,  0.20188113, -0.00582886,  0.116959  ,
        -0.227143  ,  0.10782631,  0.04656748,  0.21636172, -0.03787172,
         0.05017012, -0.22451009, -0.18570927,  0.00711007,  0.13524006,
         2.2075717 ,  2.1044874 ,  2.0182116 ]], dtype=float32)

In [233]:
print(test_y)

[[ 2.27561725 -3.34353661 -0.16820414  0.22082164  0.58329443 -1.81992401
   0.96860706  0.40624731 -3.62654111  0.83641621 -0.68401415 -0.08961869
  -0.40193849 -0.39919614  0.85355119  0.          2.          1.        ]]
